# Causal Inference: Loyalty Program Impact Analysis

This notebook demonstrates a complete causal inference workflow using [DoWhy](https://github.com/py-why/dowhy).
We estimate the causal effect of loyalty program enrollment on customer spending
using propensity score matching on simulated observational data.

> All analysis logic lives in the `src/` package — this notebook is a clean demo.

In [ ]:
import sys
sys.path.insert(0, "..")

from src.data.simulator import generate_raw_data, prepare_cohort_data
from src.models.causal import CausalAnalysis
from src.visualization.plots import plot_treatment_effect
from src.utils import load_config

## 1. Configuration

In [ ]:
cfg = load_config("../config/default.yaml")
sim = cfg["simulation"]
print(f"Simulating {sim["num_users"]:,} users over {sim["num_months"]} months")
print(f"True treatment effect: {sim["treatment_effect"]}")

## 2. Data Generation

In [ ]:
raw = generate_raw_data(
    num_users=sim["num_users"],
    num_months=sim["num_months"],
    base_spend_lambda=sim["base_spend_lambda"],
    month_decay_rate=sim["month_decay_rate"],
    treatment_effect=sim["treatment_effect"],
    seed=42,
)
raw.head(10)

In [ ]:
cohort = prepare_cohort_data(raw, signup_month=sim["signup_month"])
print(f"Cohort size: {len(cohort):,}")
print(f"Treatment: {cohort.treatment.sum():,} | Control: {(~cohort.treatment).sum():,}")
cohort.head()

## 3. Causal Model & DAG

In [ ]:
analysis = CausalAnalysis(data=cohort)
analysis.model.view_model()

from IPython.display import Image, display
display(Image(filename="../assets/causal_dag.png"))

## 4. Identification

In [ ]:
estimand = analysis.identify()
print(estimand)

## 5. Estimation

In [ ]:
estimate = analysis.estimate()
print(estimate)
print(f"
Estimated ATE: {analysis.result.ate:.4f}")
print(f"True effect:   {sim["treatment_effect"]}")

## 6. Treatment Effect Visualization

In [ ]:
fig = plot_treatment_effect(cohort)

## 7. Refutation Tests

Validate the estimate with placebo treatment, random common cause, and data subset refuters.

In [ ]:
refutations = analysis.refute()
for ref in refutations:
    print(f"{ref.name}:")
    print(f"  Original: {ref.estimated_effect:.4f}  |  New: {ref.new_effect:.4f}")
    print()